In [2]:
from tools.ner_tools import find_entities_neighbourhood, NER
import pandas as pd

In [ ]:
ner = NER()

In [ ]:
base = pd.read_json('./data/raw_articles/sampled_2022.jsonl', lines=True)

In [ ]:
base['all'] = base['lead'] + base['text']
base['entities'] = base.apply(lambda x: ner.find_entities(x['all']), axis = 1)

In [ ]:
# make df where every row is a single entity and the columns are the entity, the entity group and the article id
dfs = []
for i in range(len(base)):
    for j in range(len(base['entities'].iloc[i])):
        dfs.append(pd.DataFrame({'entity': base['entities'].iloc[i][j][0],
                        'context': base['entities'].iloc[i][j][2],
                        'label': base['entities'].iloc[i][j][1],
                        'article_id': base['id'].iloc[i]}, index=[0]))
df = pd.concat(dfs, ignore_index=True)

In [ ]:
df_text = df.merge(base[['id', 'all']], left_on='article_id', right_on='id').drop('id', axis=1)
df_text.rename(columns={'all': 'text'}, inplace=True)

In [ ]:
def find_context_in_text(context, text):
    start = text.find(context)
    end = start + len(context)
    return start, end

def process_df(
        df_to_process: pd.DataFrame,
):
    df_to_process['context'] = df_to_process.apply(lambda x: x['context'].replace(' .', '.').replace(' ,', ','), axis=1)
    df_to_process['text'] = df_to_process.apply(lambda x: x['text'].replace('\n\n', ' '), axis=1)
    df_to_process['context_start'] = df_to_process.apply(lambda x: find_context_in_text(x['context'], x['text'])[0], axis=1)
    df_to_process['context_end'] = df_to_process.apply(lambda x: find_context_in_text(x['context'], x['text'])[1], axis=1)

    df_to_process['entity_start'] = df_to_process.apply(lambda x: find_context_in_text(x['entity'], x['context'])[0] + x['context_start'], axis=1)

    df_to_process['entity_end'] = df_to_process.apply(lambda x: find_context_in_text(x['entity'], x['context'])[1] + x['context_start'], axis=1)
    df_to_process['found_entity'] = df_to_process.apply(lambda x: x['text'][x['entity_start']:x['entity_end']], axis=1)

    df_to_process = df_to_process.drop_duplicates(subset=['article_id', 'entity_end'], keep='first')
    df_to_process = df_to_process.drop_duplicates(subset=['article_id', 'entity_start'], keep='first')

    res = df_to_process[df_to_process['entity'] == df_to_process['found_entity']]
    res = res[res['entity'].apply(lambda x: len(x) > 1)]

    return res

result = process_df(df_text)